<a href="https://colab.research.google.com/github/mhco0/vis-project/blob/main/DashProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly_express
!pip install jupyter-dash


     |████████████████████████████████| 9.6 MB 7.4 MB/s 
     |████████████████████████████████| 357 kB 31.4 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=6b7fb28831c013856c734462d67235e7612856da4a19d53ff36dc0b6c0417d6a
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [2]:
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash.resources

def view_data_tips_summary(mp):
  print("-" * 8)
  print("Smokers : ")
  total_check = 0
  for smokers, value in mp['smoker'].items():
    print(".", smokers, ": ", value)
    total_check += value
  print(". Total : ", total_check)
  print("-" * 8)
  print("Days : ")
  for day, _ in mp['day'].items():
    print(".", day)
  print("-" * 8)
  print("Times : ")
  for time, _ in mp['time'].items():
    print(".", time)
  print("-" * 8)
  print("Groups : ")
  total_check = 0
  for group, value in mp['size'].items():
    print(".", group, ": ", value)
  print("-" * 8)
  print("Sex : ")
  total_check = 0
  for sex, value in mp['sex'].items():
    print(".", sex, ": ", value)
    total_check += value
  print(". Total : ", total_check)

def process_data_tips(df):
  mp = {}
  mp_day = {}
  mp_time = {}
  mp_groups = {}
  mp_sex = {"Female": 0, "Male": 0}
  mp_smokers = {"Yes": 0, "No": 0}

  for index, row in df.iterrows():
    mp_time[row['time']] = True
    mp_smokers[row['smoker']] += 1
    mp_sex[row['sex']] += 1
    if row['day'] not in mp_day:
      mp_day[row['day']] = 0
    else:
      mp_day[row['day']] += 1 

    if row['size'] not in mp_groups:
      mp_groups[row['size']] = 0
    else:
      mp_groups[row['size']] += 1 

  mp['day'] = mp_day
  mp['time'] = mp_time
  mp['size'] = mp_groups
  mp['sex'] = mp_sex
  mp['smoker'] = mp_smokers

  return mp

def blank_figure():
    fig = go.Figure(go.Scatter(x=[], y = []))
    fig.update_layout(template = None)
    fig.update_xaxes(showgrid = False, showticklabels = False, zeroline=False)
    fig.update_yaxes(showgrid = False, showticklabels = False, zeroline=False)

    return fig

# Load Data and Pre process data
df = px.data.tips()
summary = process_data_tips(df)

if __name__ == '__main__':
  numerical_columns = ['total_bill', 'tip']
  categorical_columns = [{"label": column.capitalize(), "value": column} for column in df.columns if column not in numerical_columns] 

  #print(categorical_columns)

  # Build App
  app = JupyterDash(__name__)
  
  app.layout = html.Div([
      html.Div(id="prefix", children=[
        html.H1(id='test', children="Tips Visualization", style={
        }),
        html.Br(),
        html.Label(id="output_label", children="")                         
      ], style={
          "textAlign": "center",
          "height": "10vh",
      }),

      html.Div([
                
        html.Div(id="upper_container", children=[
          dcc.Dropdown(id="selector", placeholder="Select a Categorical filter...", clearable=False, options=categorical_columns, multi=False, style={
              "width": "15vw",
          })
        ], style={
          "whiteSpace": "nowrap",
          "display": "flex"
        }),

        html.Div(id="down_container", children=[
          dcc.Graph(id="filter_graph", style={
              "width": "25vw",
              "flex": "1",
              "margin": "auto auto auto auto"
          }),
          dcc.Graph(id="total_bill_distribution", style={
              "width": "25vw",
              "flex": "1",
              "margin": "auto auto auto auto"
          }),
          dcc.Graph(id="tips_distribution", style={
              "width": "25vw",
              "flex": "1",
              "margin": "auto auto auto auto"
          }),
          dcc.Graph(id="bill_tips_relation", style={
              "width": "25vw",
              "flex": "1",
              "margin": "auto auto auto auto"
          }) 
        ], style={
          "height": "45vh",
          "whiteSpace": "nowrap",
          "display": "flex",
        })

      ], style={
          "height": "90vh"
      })
  ], style={
          "width": "80vw",
          "height": "80vh",
          "overflow": "auto"
  })

  @app.callback([
    Output(component_id='filter_graph', component_property='figure'),
    Output(component_id='total_bill_distribution', component_property='figure'),
    Output(component_id='tips_distribution', component_property='figure')
    ],
    [Input(component_id='selector', component_property='value')])
  def set_filter_graph(category):
    if category == None:
      bf = blank_figure()
      return bf, bf, bf

    if category in ['time', 'smoker', 'sex', 'day', 'size']:
      filter_histogram = px.histogram(df, x=category, facet_col_spacing=0.5, color=category, title=f"Category {category.capitalize()}", text_auto=True, labels={"y": "# Instances", category: "Instance"}).update_xaxes(type='category').update_layout(bargap=0.1)
      total_bill_boxplot = px.box(df, x=category, y='total_bill', color=category, title=f"Distribution for bills prices by {category.capitalize()}", labels={"total_bill": "Bills distribution", category: "Instance"}).update_xaxes(type='category')
      tips_boxplot = px.box(df, x=category, y='tip', color=category, title=f"Distribution for tips by {category.capitalize()}", labels={"tip": "Tips Distribution", category: "Instance"}).update_xaxes(type='category')

      return filter_histogram, total_bill_boxplot, tips_boxplot
    else:
      bf = blank_figure()
      return bf, bf, bf

  @app.callback(
    Output(component_id='bill_tips_relation', component_property='figure'),
    [Input(component_id='filter_graph', component_property='clickData')],
    [State(component_id='selector', component_property='value')])
  def filter_df_in_graphs(selectedData, category):
    if selectedData == None:
      bf = blank_figure()
      return bf
    
    if ('points' in selectedData) and len(selectedData['points']) > 0:
      if 'x' not in selectedData['points'][0]:
        bf = blank_figure()
        return bf
      
      instance = selectedData['points'][0]['x']

      if category == 'size':
        instance = int(instance)

      relation_bill_tip = px.scatter(data_frame=df[df[category] == instance], x='total_bill', y='tip', title=f"Bills X Tips [{category} with category: {instance}]", labels={"tip": "Tips in dollars", 'total_bill': "Total Bill in dollars"},  trendline='ols', trendline_color_override='red', color_discrete_sequence=['black']).update_xaxes(categoryorder='total ascending')
      return relation_bill_tip
    
    bf = blank_figure()
    return bf

  #view_data_tips_summary(summary)

  app.run_server(mode='inline', debug=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  


<IPython.core.display.Javascript object>